# Программирование на Python 

# Web-scraping: скрэйпинг сайта с книжками

*Автор: Паршина Анастасия, НИУ ВШЭ (tg: @aaparshina)*

In [4]:
import requests
from bs4 import BeautifulSoup 
from time import sleep
import pandas as pd

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/search?query=Python&page=1&sort=0&instock=&isdiscount=). Соберите информацию о книгах с первых пяти страниц. Если присмотреться, то ссылки на страницы выгядят похоже: 

    https://www.bgshop.ru/search?query=Python&page=1&sort=0&instock=&isdiscount=
    https://www.bgshop.ru/search?query=Python&page=2&sort=0&instock=&isdiscount=
    https://www.bgshop.ru/search?query=Python&page=3&sort=0&instock=&isdiscount=
    
Значит, мы можем сгенерировать необходимые ссылки сами. В таком случае чтобы узнать конечную страницу, нужно будет не нее перейти и посмотреть номер. Что, конечно не супер автоматизировано, но так мы точно не ошибемся. 

Другой вариант действий: в коде страницы есть ссыка на "Следующую" страницу. Можно собирать именно их *до тех пор, пока* они не закончатся (тут вам в помощь цикл `while`). Как узнать номер страницы? Посмотреть на ее код!

In [5]:
url = 'https://www.bgshop.ru/search?query=Python&page=1&sort=0&instock=&isdiscount='
page = requests.get(url)
soup = BeautifulSoup(page.text)
soup.find_all('a', attrs = {'aria-label': 'Next'})[-1]

<a aria-label="Next" class="page-link" href="search?query=Python&amp;page=13&amp;sort=0&amp;instock=&amp;isdiscount="><span aria-hidden="true">»»</span><span class="sr-only">Next</span></a>

Это наша последняя страница. Заберем оттуда ссылку:

In [6]:
num = soup.find_all('a', attrs = {'aria-label': 'Next'})[-1].get('href').split('page=')[1].split('&')[0]
print(num)

13


In [7]:
urls = []

for i in range(1, int(num) + 1):
    urls.append(f'https://www.bgshop.ru/search?query=Python&page={i}&sort=0&instock=&isdiscount=')
    
urls

['https://www.bgshop.ru/search?query=Python&page=1&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=2&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=3&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=4&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=5&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=6&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=7&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=8&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=9&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=10&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=11&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/search?query=Python&page=12&sort=0&instock=&isdiscount=',
 'https://www.bgshop.ru/s

Разберем все на примере одной страницы. 

In [8]:
page = requests.get(urls[0])
soup = BeautifulSoup(page.text)

Заберем ссылки на все книжки на странице. 

In [9]:
urls_books = ['https://www.bgshop.ru' + i.get('href') \
              for i in soup.find_all('a', attrs = {'class':'img_link'})]
urls_books


['https://www.bgshop.ru/Catalog/GetFullDescription?id=10483871&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10499950&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10224348&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10645085&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10627655&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10473444&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10901801&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10891776&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10879726&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10873807&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10865883&type=1',
 'https://www.bgshop.ru/Catalog/GetFullDescription?id=10831874&type=1']

Посмотрим на одну книжку: 

In [10]:
url0 = urls_books[0]
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

Найдем основную информацию со страницы

In [11]:
all_info = soup0.find('div', attrs = {'class': 'item-desc'})
all_info

<div class="item-desc">
<div id="rating-wrapper">
<div class="starrr" data-target="#login-modal" data-toggle="modal">
<a aria-hidden="true" class="fa fa-star-o" href="#"></a>
<a aria-hidden="true" class="fa fa-star-o" href="#"></a>
<a aria-hidden="true" class="fa fa-star-o" href="#"></a>
<a aria-hidden="true" class="fa fa-star-o" href="#"></a>
<a aria-hidden="true" class="fa fa-star-o" href="#"></a>
</div>
</div><span data-target="#login-modal" data-toggle="modal" id="vote-qty">Количество оценок: <span>0</span></span>
<div>
<span class="your-choice-was" style="display: none;">
<span class="choice"></span>
</span>
</div>
<h1 id="p_title_10483871">Глубокое обучение на Python</h1>
<p class="goToDescription" style="margin-bottom:0;">
<a class="scroll-to" href="/Catalog/Search?query=Шолле Ф.&amp;page=1">
Франсуа Шолле            </a>
</p>
<div style="color:#777777; font-size:.8rem; padding:12px 0; display:none;">Заказали за последний месяц: 1 человек</div>
<table class="decor2">
<tr>
<td cl

Выше мы ищем тег `div`, при этом уточним, что у этого тега есть атрибут `class` со значением `item-desc`.

In [12]:
title = all_info.find('h1').text
title

'Глубокое обучение на Python'

In [13]:
author = all_info.find('a', attrs = {'class': 'scroll-to'}).text.strip()
author

'Франсуа Шолле'

Обратим внимание на статус - "в наличии". Кусочек кода выглядит так:

    <p class="item-status">в наличии <span style="color: #777777; font-size: .8rem; display:none;">(остаток: 11 шт.)</span></p>
    
НО! Есть книжки, которых нет в наличии. И у них эта же часть выглядит так: 

    <p class="item-status"></p>

In [14]:
status = soup0.find('p', attrs = {'class': 'item-status'}).text
status

'в наличии (остаток: 14 шт.)'

Можно также попробовать вытащить аннотацию.

In [15]:
ann = soup0.find('div', attrs = {'id' : 'collapseExample'}).text.split('\n')
ann = '\n'.join([a.strip('• \r') for a in ann if len(a) > 100])
ann

'Глубокое обучение — Deep learning — это набор алгоритмов машинного обучения, которые моделируют высокоуровневые абстракции в данных, используя архитектуры, состоящие из множества нелинейных преобразований. Согласитесь, эта фраза звучит угрожающе. Но всё не так страшно, если о глубоком обучении рассказывает Франсуа Шолле, который создал Keras — самую мощную библиотеку для работы с нейронными сетями. Познакомьтесь с глубоким обучением на практических примерах из самых разнообразных областей. Книга делится на две части, в первой даны теоретические основы, вторая посвящена решению конкретных задач. Это позволит вам не только разобраться в основах DL, но и научиться использовать новые возможности на практике.\nОбучение — это путешествие длинной в жизнь, особенно в области искусственного интеллекта, где неизвестностей гораздо больше, чем определенности.'

И цену книжки:
    

In [16]:
price = float(soup0.find('div', attrs = {'id': 'price'}).text.split()[-2].replace(',', '.'))
price

1739.0

Сделаем функцию для обработки каждой страницы

In [17]:
def GetBookInfo(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    all_info = soup0.find('div', attrs = {'class': 'item-desc'})
    
    title = all_info.find('h1').text 
    
    author = all_info.find('a', attrs = {'class': 'scroll-to'}).text.strip()
    
    status = soup0.find('p', attrs = {'class': 'item-status'}).text

    ann = soup0.find('div', attrs = {'id' : 'collapseExample'}).text.split('\n')
    ann = '\n'.join([a.strip('• \r') for a in ann if len(a) > 100])
    
    price = float(soup0.find('div', attrs = {'id': 'price'}).text.split()[-2].replace(',', '.'))

        
    return url0, title, author, status, ann, price

In [18]:
GetBookInfo(url0)

('https://www.bgshop.ru/Catalog/GetFullDescription?id=10483871&type=1',
 'Глубокое обучение на Python',
 'Франсуа Шолле',
 'в наличии (остаток: 14 шт.)',
 'Глубокое обучение — Deep learning — это набор алгоритмов машинного обучения, которые моделируют высокоуровневые абстракции в данных, используя архитектуры, состоящие из множества нелинейных преобразований. Согласитесь, эта фраза звучит угрожающе. Но всё не так страшно, если о глубоком обучении рассказывает Франсуа Шолле, который создал Keras — самую мощную библиотеку для работы с нейронными сетями. Познакомьтесь с глубоким обучением на практических примерах из самых разнообразных областей. Книга делится на две части, в первой даны теоретические основы, вторая посвящена решению конкретных задач. Это позволит вам не только разобраться в основах DL, но и научиться использовать новые возможности на практике.\nОбучение — это путешествие длинной в жизнь, особенно в области искусственного интеллекта, где неизвестностей гораздо больше, чем 

Пробуем нашу функцию применить. 

In [19]:
books = []
for u in urls[:5]:
    page0 = requests.get(u)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    
    # если вы были на паре, то помните, что тут было проще —
    # просто я хотела 1) напомнить, что можно использовать списковые включения; 
    # 2) про перенос кода с помощью \
    
    urls_books = ['https://www.bgshop.ru' + i.get('href') \
              for i in soup.find_all('a', attrs = {'class':'img_link'})]
    
    for i in urls_books:
        print(i) # печатаем ссылки, что просто понимать, что у нас ничего не сломалось и все работает 
        res = GetBookInfo(i)
        books.append(res)
        sleep(1)

https://www.bgshop.ru/Catalog/GetFullDescription?id=10483871&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10499950&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10224348&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10645085&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10627655&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10473444&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10901801&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10891776&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10879726&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10873807&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10865883&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10831874&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10483871&type=1
https://www.bgshop.ru/Catalog/GetFullDescription?id=10499950&type=1
https://www.bgshop.ru/Catalog/GetFullDescription

In [20]:
df = pd.DataFrame(books)
df.columns = ['url', 'title', 'author', 'status', 'ann', 'price']
df.head(4)

,url,title,author,status,ann,price
0,https://www.bgshop.ru/Catalog/GetFullDescripti...,Глубокое обучение на Python,Франсуа Шолле,в наличии (остаток: 14 шт.),Глубокое обучение — Deep learning — это набор ...,1739.0
1,https://www.bgshop.ru/Catalog/GetFullDescripti...,Чистый Python. Тонкости программирования для п...,Дэн Бейдер,в наличии (остаток: 11 шт.),Изучение всех возможностей Python — сложная за...,1529.0
2,https://www.bgshop.ru/Catalog/GetFullDescripti...,Программирование на Python для начинающих,Майк МакГрат,в наличии (остаток: 15 шт.),"Книга ""Программирование на Python для начинаю...",849.0
3,https://www.bgshop.ru/Catalog/GetFullDescripti...,Легкий способ выучить Python 3 еще глубже,Зед Шоу,в наличии (остаток: 11 шт.),"Зед Шоу – один из тех, кто по-настоящему разби...",1029.0


In [21]:
df.to_csv('books.csv')